In [ ]:
from env.core import FunctionExpr, Param
from env.full_state import FullState
from env.meta_env import MetaInfo
from env.goal_env import GoalEnv
from env.node_types import HaveDefinition, AndNode, OrNode, IntBooleanNode

params = (Param.from_int(1), Param.from_int(2), Param.from_int(3))
p1, p2, p3 = params
goal = HaveDefinition(
    FunctionExpr.with_child(
        OrNode(
            AndNode(p1, p2, IntBooleanNode(1)),
            AndNode(p2, p3),
        ),
    )
)

env = GoalEnv(goal)
node_types = env.full_state.node_types()
print('Amount of node types:', len(node_types))

In [2]:
selected_goal = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_goal)).apply()
assert selected_goal == goal

In [ ]:
GoalEnv.symbolic(selected_goal, node_types)

In [ ]:
GoalEnv.data_array(selected_goal, node_types)

In [ ]:
env.to_symbolic_current_state()

In [ ]:
env.to_data_array_current_state()

In [ ]:
import typing
print(typing.get_origin(FunctionExpr))
print(typing.get_origin(FunctionExpr[str]))
print(typing.get_origin('FunctionExpr'))

In [8]:
# env.to_symbolic()

In [9]:
env.full_state.validate()